William Andrés Gómez Roa

Punto 2

Parcial 3

In [1]:
install.packages("gbm")
library(gbm)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loaded gbm 2.2.2

This version of gbm is no longer under development. Consider transitioning to gbm3, https://github.com/gbm-developers/gbm3



In [14]:
install.packages("caret")
library(caret)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘future’, ‘future.apply’, ‘diagram’, ‘lava’, ‘prodlim’, ‘proxy’, ‘iterators’, ‘clock’, ‘hardhat’, ‘ipred’, ‘e1071’, ‘foreach’, ‘ModelMetrics’, ‘plyr’, ‘pROC’, ‘recipes’, ‘reshape2’


Loading required package: ggplot2

Loading required package: lattice



In [15]:
#DATASET LOADING
Hospital_data<-read.csv("Datos_Hospital.csv", sep=";")

#PREPROCESSING
Hospital_data$AgeYrs <- as.numeric(Hospital_data$AgeYrs)
Hospital_data$Estimatedcost <- as.numeric(Hospital_data$Estimatedcost)
Hospital_data$NPS_Status <- as.factor(Hospital_data$NPS_Status)
Hospital_data <- Hospital_data[!is.na(Hospital_data$AgeYrs) & !is.na(Hospital_data$Estimatedcost), ]
Hospital_data$NPS_Status <- as.numeric(Hospital_data$NPS_Status) - 1
str(Hospital_data)

# TRAIN and TEST
set.seed(8837)

indices <- sample(1:nrow(Hospital_data), size = 0.65 * nrow(Hospital_data))
train_data <- Hospital_data[indices, ]  # Conjunto de entrenamiento
test_data <- Hospital_data[-indices, ]


#FOR PREDICTING
true_labels <- as.factor(test_data$NPS_Status)

Warning message:
“NAs introduced by coercion”
Warning message:
“NAs introduced by coercion”


'data.frame':	3562 obs. of  11 variables:
 $ SN                  : int  1001 1002 1003 1004 1005 1006 1007 1008 1009 1010 ...
 $ AgeYrs              : num  33 12 3 63 27 50 45 22 83 29 ...
 $ Estimatedcost       : num  70000 30000 75000 40000 25000 60000 30000 30000 50000 25000 ...
 $ LengthofStay        : int  33 2 1 6 2 7 1 2 4 2 ...
 $ InsPayorcategory    : int  2 5 5 3 5 3 3 5 5 3 ...
 $ CE_VALUEFORMONEY    : int  2 3 3 3 3 2 2 3 3 3 ...
 $ CE_CSAT             : int  2 3 3 3 3 3 2 3 3 3 ...
 $ AE_PATIENTSTATUSINFO: int  4 4 3 3 4 4 3 3 4 4 ...
 $ CE_ACCESSIBILITY    : int  3 4 3 4 4 4 3 4 3 4 ...
 $ CE_NPS              : int  7 7 8 8 7 5 5 7 7 8 ...
 $ NPS_Status          : num  0 0 0 0 0 0 0 0 0 0 ...


In [27]:
#TRAINING: BASE
set.seed(8837)

gbm_base <- gbm(
  formula = NPS_Status ~ .,
  data = train_data,
  distribution = "bernoulli",  # Clasificación binaria
  n.trees = 5000,             # Número de árboles
  interaction.depth = 4,      # Profundidad máxima de cada árbol
  shrinkage = 0.01,           # Tasa de aprendizaje
  verbose = FALSE             # No imprimir mensajes
)


# PREDICT
yhat_base <- predict(gbm_base, newdata = test_data, n.trees = 5000, type = "response")
pred_base <- ifelse(yhat_base >= 0.1, 1, 0)
conf_base <- confusionMatrix(as.factor(pred_base), true_labels)

#PRINT
cat("Confusion Matrix for Base Model:\n")
print(conf_base)

Confusion Matrix for Base Model:
Confusion Matrix and Statistics

          Reference
Prediction   0   1
         0 616   0
         1   0 631
                                    
               Accuracy : 1         
                 95% CI : (0.997, 1)
    No Information Rate : 0.506     
    P-Value [Acc > NIR] : < 2.2e-16 
                                    
                  Kappa : 1         
                                    
 Mcnemar's Test P-Value : NA        
                                    
            Sensitivity : 1.000     
            Specificity : 1.000     
         Pos Pred Value : 1.000     
         Neg Pred Value : 1.000     
             Prevalence : 0.494     
         Detection Rate : 0.494     
   Detection Prevalence : 0.494     
      Balanced Accuracy : 1.000     
                                    
       'Positive' Class : 0         
                                    


## **-** Se creo y entreno un modelo de Random Forest llamado rf_base ("modelo rf base")

## **1.** ¿Cómo se podría obtener un sobre-ajuste en el modelo GB base? Si es posible, aplíquelo y demuestre que hay un sobre-ajuste. Si no es posible, explique brevemente porqué.

In [28]:
# TRAINING: SOBREAJUSTE
set.seed(8837)

gbm_sob <- gbm(
  formula = NPS_Status ~ .,
  data = train_data,
  distribution = "bernoulli",  # Clasificación binaria
  n.trees = 10000,            # Más árboles
  interaction.depth = 6,      # Mayor profundidad
  shrinkage = 0.001,          # Tasa de aprendizaje más baja
  verbose = FALSE             # No imprimir mensajes
)

# PREDICCIONES
yhat_sob <- predict(gbm_sob, newdata = test_data, n.trees = 10000, type = "response")
pred_sob <- ifelse(yhat_sob >= 0.1, 1, 0)
conf_sob <- confusionMatrix(as.factor(pred_sob), true_labels)
#PRINT
cat("\nConfusion Matrix for Overfitting Model:\n")
print(conf_sob)


Confusion Matrix for Overfitting Model:
Confusion Matrix and Statistics

          Reference
Prediction   0   1
         0 616   0
         1   0 631
                                    
               Accuracy : 1         
                 95% CI : (0.997, 1)
    No Information Rate : 0.506     
    P-Value [Acc > NIR] : < 2.2e-16 
                                    
                  Kappa : 1         
                                    
 Mcnemar's Test P-Value : NA        
                                    
            Sensitivity : 1.000     
            Specificity : 1.000     
         Pos Pred Value : 1.000     
         Neg Pred Value : 1.000     
             Prevalence : 0.494     
         Detection Rate : 0.494     
   Detection Prevalence : 0.494     
      Balanced Accuracy : 1.000     
                                    
       'Positive' Class : 0         
                                    


**Sobreajuste:** Se obtuvo ajustando n.trees = 10000, interaction.depth = 6 y shrinkage = 0.001. Esto implica que el modelo es excesivamente complejo, ajustándose demasiado a los datos de entrenamiento.

## **2.** ¿Cómo se podría obtener un sub-ajuste en el modelo GB base? Si es posible, aplíquelo y demuestre que hay un subajuste. Si no es posible, explique brevemente porqué.

In [29]:
# TRAINING: SUBAJUSTE
set.seed(8837)

gbm_sub <- gbm(
  formula = NPS_Status ~ .,
  data = train_data,
  distribution = "bernoulli",  # Clasificación binaria
  n.trees = 10,               # Menos árboles
  interaction.depth = 1,      # Árboles más simples
  shrinkage = 0.1,            # Tasa de aprendizaje alta
  verbose = FALSE             # No imprimir mensajes
)

# PREDICCIONES
yhat_sub <- predict(gbm_sub, newdata = test_data, n.trees = 50, type = "response")
pred_sub <- ifelse(yhat_sub >= 0.1, 1, 0)
conf_sub <- confusionMatrix(as.factor(pred_sub), true_labels)
#PRINT
cat("\nConfusion Matrix for Underfitting Model:\n")
print(conf_sub)

Warning message in predict.gbm(gbm_sub, newdata = test_data, n.trees = 50, type = "response"):
“Number of trees not specified or exceeded number fit so far. Using 10.”
Warning message in confusionMatrix.default(as.factor(pred_sub), true_labels):
“Levels are not in the same order for reference and data. Refactoring data to match.”



Confusion Matrix for Underfitting Model:
Confusion Matrix and Statistics

          Reference
Prediction   0   1
         0   0   0
         1 616 631
                                          
               Accuracy : 0.506           
                 95% CI : (0.4779, 0.5341)
    No Information Rate : 0.506           
    P-Value [Acc > NIR] : 0.5113          
                                          
                  Kappa : 0               
                                          
 Mcnemar's Test P-Value : <2e-16          
                                          
            Sensitivity : 0.000           
            Specificity : 1.000           
         Pos Pred Value :   NaN           
         Neg Pred Value : 0.506           
             Prevalence : 0.494           
         Detection Rate : 0.000           
   Detection Prevalence : 0.000           
      Balanced Accuracy : 0.500           
                                          
       'Positive' Class : 0    

**Subajuste:** Se obtuvo ajustando n.trees = 10, interaction.depth = 1 y shrinkage = 0.1. Esto implica que el modelo es excesivamente simple, ajustándose pobremente a los datos de entrenamiento.

** RESPUESTA FINAL: ** Se pudo observar que es posible sobreajustar o subajustar el modelo *gbm*, por medio de la modificación de algúnos parametros como el número de árboles, la profundidad de cada árbol o la tása de aprendizaje.

Sin embargo se ha observado que la variable que es de interes predecir `NPS_Status` esta altamente correlacionada con algunas de sus variable spredictoras, practicamente revelando la respuesta. Esto puede estar generando que los modelos BOOST especificamente el *gbm* tengan una gran facilidad para predecir correctamente.

Por lo tanto, aquí se ha modificado el umbral de prediccion a 0.1, con el fin de observar algún cambio (umbral=0.5 no genera diferencia enlos 3 modelos). Pudimos observar finalmente, que con un umbral de 0.1 el modelo con **subajuste** el modelo predice todos los datos como si fueran de la clase **1**.